# Líneas base de Google EIE

Imágenes por estación de los cálculos de potencial solar, emisiones de edificios y emisiones de transporte con datos de Google EIE

### Ajustes generales

In [2]:
import os
import sys
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import osgeo.gdal
import urllib.request
import zipfile
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aqiGDL
%matplotlib inline

In [3]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
colors = ['7A76C2', 'ff6e9c98', 'f62196', '18c0c4', 'f3907e', '66E9EC']
dist = 1000 #Catchment area in sq m.

In [4]:
gdf_est = aqiGDL.gdf_from_db('estaciones_gdl','Estaciones')
#gdf_est = ox.project_gdf(gdf_est,to_crs='EPSG:32613')

## Potencial solar

In [5]:
potential = (6600/28) #Per roof median potential kWh AC/yr https://insights.sustainability.google/places/ChIJOwV0Q_qxKIQR7NCkjDwfR-k/solar

In [6]:
data = []
areas = {}
edges_data = []
for i in range(len(gdf_est)):
    x = gdf_est.at[i,'x']
    y = gdf_est.at[i,'y']
    est = gdf_est.at[i,'Name']
    point = (y, x)
    tags = {'building': True}
    gdf = ox.geometries_from_point(point, tags, dist=dist)
    gdf['Estacion'] = est
    # calculate the area in projected units (meters) of each building footprint
    gdf = ox.project_gdf(gdf)
    gdf['area'] = gdf.area
    gdf['kWh_year'] = gdf['area'] * potential
    data.append(gdf)
    areas[est] = gdf['area'].sum()
    G = ox.graph_from_point(point,dist=dist)
    edges = ox.graph_to_gdfs(G, nodes=False)
    edges['Estacion'] = est
    edges = ox.project_gdf(edges)
    edges_data.append(edges)
    G = None
    edges = None

In [7]:
df = pd.DataFrame(areas, index = ['area']).T
df['kWh_year'] = df['area'] * potential
gdf_est_pot = ox.project_gdf(gdf_est)
gdf_est_pot = gdf_est_pot.merge(df, right_index=True, left_on = 'Name')

In [8]:
gdf_ = pd.concat(data, ignore_index=True)
edges = pd.concat(edges_data, ignore_index=True)

## Emisiones de edificioes

In [9]:
consumed = 144.06 #Per roof median emission kWh/m²/yr https://insights.sustainability.google/places/ChIJOwV0Q_qxKIQR7NCkjDwfR-k/buildings
emission = 0.00041423 #total carbon intensity tCO2e/kWh https://insights.sustainability.google/places/ChIJOwV0Q_qxKIQR7NCkjDwfR-k/buildings

Descarga y descomprime los datos para el DENUE 2019

In [10]:
save_path = os.path.join('../data/external/', 'denue')
urllib.request.urlretrieve(
            'https://www.inegi.org.mx/contenidos/masiva/denue/2019_11/denue_14_1119_shp.zip', save_path)


# define the name of the directory to be created
outputDirectory = '../data/external/denue_2019/'

try:
    os.makedirs(outputDirectory)
except IOError:
    print('cannot create', outputDirectory)
    
with zipfile.ZipFile(save_path, 'r') as zip_ref:
    
    zip_ref.extractall(outputDirectory)

cannot create ../data/external/denue_2019/


In [11]:
denue = gpd.read_file(outputDirectory+'conjunto_de_datos/denue_inegi_14_.shp')

denue = ox.project_gdf(denue,to_crs='EPSG:32613')

Crea el buffer para filtrar los puntos del DENUE por área de estudio

In [12]:
gdf_est = ox.project_gdf(gdf_est,to_crs='EPSG:32613')
buffer = gpd.GeoDataFrame({'geometry':gdf_est.buffer(dist),'Name':gdf_est['Name']},geometry='geometry',crs=gdf_est.crs)

clip = gpd.clip(denue, buffer)
clip.head(1)

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta,geometry
10,7162426,TOTAL PUERTAS AUTOMATICAS,None,238290,Otras instalaciones y equipamiento en construc...,0 a 5 personas,CALLE,VOLCAN PARICUTIN,CALLE,VOLCAN DE BARU,...,4431,003,None,None,None,Fijo,20.643058,-103.432993,2019-11,POINT (663256.618 2283431.913)


In [13]:
gob = clip[clip['codigo_act'].str.startswith('931')]
gob.head(1)

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta,geometry
361126,1694541,ALMACÉN GENERAL DE GOBIERNO,ALMACÉN GENERAL DE GOBIERNO,931210,Administración pública en general,0 a 5 personas,CALLE,PUERTO GUAYMAS,CALLE,ANILLO PERIFERICO PONIENTE MANUEL GOMEZ MORIN,...,1723,017,3030954041,None,None,Fijo,20.641361,-103.441811,2014-12,POINT (662339.540 2283235.202)


Extrae las áreas de los edificios en las áreas de estudio para obtener el área promedio

In [14]:
buffer = ox.project_gdf(buffer,to_crs='EPSG:4326')
tags = {'building': True}
gdf_building = gpd.GeoDataFrame()

for b in buffer.geometry:
    building = ox.geometries_from_polygon(b, tags)
    gdf_building = gdf_building.append(building)

In [15]:
gdf_building = ox.project_gdf(gdf_building,to_crs='EPSG:32613')
gdf_building['area'] = gdf_building.geometry.area
mean_area = gdf_building.area.mean()
mean_area

2122.6634299477205

Cálculo del área y emisiones promedio por estación

In [16]:
buffer = ox.project_gdf(buffer,to_crs='EPSG:32613')

edificios_gob = gpd.sjoin(buffer,gob,how='left').groupby(['Name']).count().reset_index()
edificios_gob = edificios_gob[['Name','id']]

edificios_gob['area'] = edificios_gob['id']*mean_area
edificios_gob['kWh_year'] = edificios_gob['area']*consumed
edificios_gob['tonCO₂eq'] = edificios_gob['kWh_year']*emission
edificios_gob.head(5)

,Name,id,area,kWh_year,tonCO₂eq
0,1. Terminal Sur,2,4245.32686,6.115818e+05,253.335524
1,10. Colon,4,8490.65372,1.223164e+06,506.671048
2,11. UVM,3,6367.99029,9.173727e+05,380.003286
3,12. ITESO,0,0.00000,0.000000e+00,0.000000
4,13. Lopez Mateos,5,10613.31715,1.528954e+06,633.338810


In [17]:
edificios_gob_ = edificios_gob.merge(gdf_est, right_on='Name', left_on = 'Name')
edificios_gob_ = gpd.GeoDataFrame(edificios_gob_, crs = gdf_est.crs)
edificios_gob_.sort_values(by='tonCO₂eq', ascending=False, inplace=True)
edificios_gob_.head(1)

,Name,id,area,kWh_year,tonCO₂eq,x,y,geometry
30,37. Tabachines,15,31839.951449,4.586863e+06,1900.016429,-103.362075,20.733951,POINT (670544.948 2293566.891)


## Emisiones de transporte

In [18]:
gdf_amg = aqiGDL.gdf_from_db('municipios_amg','areas')
gdf_amg = ox.project_gdf(gdf_amg,to_crs='EPSG:32613')
gdf_amg.head(1)

,CVE_ENT,CVE_MUN,NOM_MUN,OID,geometry
0,14,039,Guadalajara,631,"MULTIPOLYGON (((675777.419 2295506.182, 675934..."


In [19]:
df_cd = pd.read_csv('../data/external/Google-emisiones-transporte.csv')
df_cd.Ciudad.apply(str)
df_cd.head(1)

,Ciudad,Modo,Porcentaje-uso,Eficiencia,Emision
0,Guadalajara,Automovil,85.6,9.1,0.002


In [20]:
gdf_est['municipio'] = gpd.sjoin(gdf_est,gdf_amg)['NOM_MUN']
gdf_est.head(1)

,Name,x,y,geometry,municipio
0,10. Colon,-103.400806,20.606105,POINT (666650.899 2279373.835),San Pedro Tlaquepaque


In [21]:
gdf_edge = aqiGDL.gdf_from_db('guadalajara_edges','networks')
gdf_edge = ox.project_gdf(gdf_edge,to_crs='EPSG:32613')
gdf_edge.head(1)

,osmid,highway,oneway,length,geometry,name,lanes,bridge,access,junction,maxspeed,ref,service,tunnel,width,area,u,v,key
0,149698947,secondary,True,51.862,"LINESTRING (678752.310 2282699.577, 678738.504...",Avenida Patria,2,None,None,None,None,None,None,None,None,None,1619378249,4746157497,0


In [22]:
for i in range(len(buffer)):
    vialidades = gpd.clip(gdf_edge, buffer.iloc[i].geometry)
    vialidades['length'] = clip.geometry.length
    
    buffer.loc[i,'sum_length'] = vialidades.length.sum()

In [23]:
buffer = buffer.merge(gdf_est.drop(column=['geometry','x','y']), right_on='Name', left_on = 'Name')
buffer = gpd.GeoDataFrame(buffer)
buffer.head(1)

,geometry_x,Name,sum_length,x,y,geometry_y,municipio
0,"POLYGON ((667650.899 2279373.835, 667646.084 2...",10. Colon,100792.772838,-103.400806,20.606105,POINT (666650.899 2279373.835),San Pedro Tlaquepaque


In [27]:
for c in df_cd.Ciudad:
    
    buffer_c = buffer.loc[buffer.municipio==c]
    
    for est in buffer_c.Name:
    
        distancia = float(buffer_c.loc[buffer_c.Name==est]['sum_length']/1000)
        
        #print ()
        
        co2_auto = (distancia*float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Porcentaje-uso']/100)/
                    float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Eficiencia'])*
                   float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Emision']))
        
        co2_bus = (distancia*float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Autobus'),'Porcentaje-uso']/100)/
                    float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Eficiencia'])*
                   float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Emision']))
        
        co2_moto = (distancia*float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Motocicleta'),'Porcentaje-uso']/100)/
                    float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Eficiencia'])*
                   float(df_cd.loc[(df_cd.Ciudad==c)&(df_cd.Modo=='Automovil'),'Emision']))
        
        buffer.loc[(buffer.Name==est), 'tonCO₂eq'] = (co2_auto + co2_bus + co2_moto) * 254

In [35]:
gdf_est_tpt = gpd.sjoin(gdf_est,buffer.drop(columns=['Name']))
gdf_est_tpt = gdf_est_tpt.drop_duplicates(subset = 'Name', keep = 'first')
gdf_est_tpt.head(1)

,Name,x,y,geometry,municipio_left,index_right,sum_length,municipio_right,tonCO₂eq
0,10. Colon,-103.400806,20.606105,POINT (666650.899 2279373.835),San Pedro Tlaquepaque,1,89913.773403,San Pedro Tlaquepaque,5.356878


In [36]:
G = ox.graph_from_bbox(20.751857,20.523110,-103.201328,-103.468643)
edges_amg = ox.graph_to_gdfs(G, nodes=False)
edges_amg = ox.project_gdf(edges_amg,to_crs=gdf_est.crs)

In [37]:
errors = []
for est in gdf_est.Name.unique():
    fig, axes = plt.subplots(2,2,figsize=(24,14))
    #ax = 
    
    #emisiones de transporte
    ax = axes[0,1]
    
    edges[edges['Estacion'] == est].plot(ax=ax, color='w', zorder=1, alpha=0.25, linewidth=1)
    gdf_est_tpt[gdf_est_tpt['Name'] == est].plot(ax=ax, column ='tonCO₂eq', cmap='YlOrRd', markersize=gdf_est_tpt[gdf_est_tpt['Name'] == est]['tonCO₂eq']*500, alpha=0.85, zorder=3, vmin=gdf_est_tpt['tonCO₂eq'].min(), vmax=gdf_est_tpt['tonCO₂eq'].max())
    ax.set_title('{}\n{:,} tonCO2eq anual'.format('Emisiones de transporte', round(gdf_est_tpt[gdf_est_tpt['Name'] == est]['tonCO₂eq'].values[0],2)), fontsize=15)
    ax.axis('off')
    #ax.set_visible(False)
    
    #emisiones de edificios
    ax=axes[1,0]
    
    #est = edificios_gob_.at[i,'Name']
    edges[edges['Estacion'] == est].plot(ax=ax, color='w', zorder=1, alpha=0.25, linewidth=1)
    edificios_gob_[edificios_gob_['Name'] == est].plot(ax=ax, column ='tonCO₂eq', cmap='YlOrRd', markersize=edificios_gob_[edificios_gob_['Name'] == est]['tonCO₂eq']*20, alpha=0.85, zorder=3, vmin=edificios_gob_['tonCO₂eq'].min(), vmax=edificios_gob_['tonCO₂eq'].max())
    gob.plot(ax=ax, color='#7A76C2', zorder=2)
    
    minx, miny, maxx, maxy = edges.loc[edges.Estacion==est].geometry.total_bounds
    ax.set_xlim(minx - .1, maxx + .1) # added/substracted value is to give some margin around total bounds
    ax.set_ylim(miny - .1, maxy + .1)
    
    ax.set_title('{}\n{:,} tonCO2eq anuales'.format('Emisiones de edificios', round(edificios_gob_[edificios_gob_['Name'] == est]['tonCO₂eq'].values[0],2)), fontsize=15)
    ax.axis('off')
    
    #potencial solar
    ax = axes[1,1]
        
    gdf_[gdf_['Estacion'] == est].plot(ax=ax, color='#7A76C2', zorder=2)
    edges[edges['Estacion'] == est].plot(ax=ax, color='w', zorder=1, alpha=0.25, linewidth=1)
    gdf_est_pot[gdf_est_pot['Name'] == est].plot(ax=ax, column ='kWh_year', cmap='YlOrRd', markersize=gdf_est_pot[gdf_est_pot['Name'] == est]['kWh_year']/50000, alpha=0.85, zorder=3, vmin=gdf_est_pot['kWh_year'].min(), vmax=gdf_est_pot['kWh_year'].max())
    ax.set_title('{}\n{:,} kWh anuales'.format('Potencial solar', round(gdf_est_pot[gdf_est_pot['Name'] == est]['kWh_year'].values[0],2)), fontsize=15)
    ax.axis('off')
    
            
    a00 = axes[0,0]
    shax = a00.get_shared_x_axes()
    shax.remove(a00)
    a00.set_title('Ubicación', fontsize=15)
    edges_amg.plot(ax=a00, color='#e8e9eb',linewidth=0.1, zorder=-1)
    edges_amg[(edges_amg['highway']=='primary') | (edges_amg['highway']=='secondary')].plot(ax=a00, color='#e8e9eb',linewidth=0.5, zorder=0)
    gdf_est.plot(ax=a00, color='k', alpha=0.85, zorder=1)
    gdf_est[gdf_est['Name']==est].plot(ax=a00, color='#ba0d38', alpha=0.85, zorder=2, markersize=150)
    a00.axis('off')
    
    name_est = est.split(' ',maxsplit=1)[1]
    
    fig.suptitle(f'Estación {name_est}\n Línea base con Google EIE', fontsize=30)
    plt.savefig(f'../output/figures/google_eie/{est}_GoogleEIE.png',dpi=300)
    plt.savefig(f'../output/figures/google_eie/{est}_GoogleEIE.svg',dpi=300)
    plt.close()

IndexError: single positional indexer is out-of-bounds